In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../data/answerList_data.csv')
df

,Answer.ID,FailingMethod,Question.ID,Answer.duration,Answer.confidence,Answer.difficulty,GroundTruth,TP,TN,FN,...,Code.complexity,Worker.ID,Worker.score,Worker.profession,Worker.yearsOfExperience,Worker.age,Worker.gender,Worker.whereLearnedToCode,Worker.country,Worker.programmingLanguage
0,261,HIT01_8,0,90.984,4,2,0,0,1,0,...,1,832cg-7G1i-462:73eI-8E-2g-985,5,Undergraduate_Student,7.0,21,Male,High School,United States,Java; C++; C#
1,262,HIT01_8,0,133.711,5,1,0,0,1,0,...,1,98ce7A-4i-507,4,Undergraduate_Student,10.0,25,Female,High School;University;Web,United States,c#
2,263,HIT01_8,0,77.696,5,2,0,0,1,0,...,1,881AC0I2E-625:135cI3E-7e8-86,5,Professional_Developer,7.0,24,Male,High School;University;Web,United States,C++;Java;PHP
3,264,HIT01_8,0,46.644,1,1,0,0,1,0,...,1,270eG-5c6A3-46:65eI-2i-2I-9-36,5,Professional_Developer,20.0,38,Male,High School;University;Other On the job,USA,C#
4,265,HIT01_8,0,215.416,5,5,0,0,1,0,...,1,844Cg1g5e1-17:255ei-8G8E85-8,3,Undergraduate_Student,5.0,19,Male,High School;University;Web,United States,C++; Java
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,2316,HIT08_54,128,220.420,2,4,0,0,1,0,...,1,1171ea-4g-6C-73-8,3,Graduate_Student,2.0,28,Male,University,USA,C#
2576,2317,HIT08_54,128,322.790,4,3,0,0,0,0,...,1,66AC-5a0g-47-9:1443IA-7C-6e967,5,Professional_Developer,17.0,39,Male,University;Web,USA,C#; VB.NET; Java
2577,2318,HIT08_54,128,159.530,5,1,0,0,1,0,...,1,106iG8G-9I-9-80:590CG-6G-7i-71-9,4,Professional_Developer,10.0,31,Male,High School;University;Web;Other work,usa,C++
2578,2319,HIT08_54,128,68.578,5,1,0,0,1,0,...,1,1221iC8A5A242:495CC9e6a691:11aE2c-4c-9-86,4,Undergraduate_Student,4.0,19,Male,University;Web;Other FIRST Robotics,United States,C++


In [3]:
df['correctness'] = (((df['GroundTruth'] == 0.0) & (df['Answer.option'] == 'NO')) | (
            (df['GroundTruth'] == 1.0) & (df['Answer.option'] == 'YES'))).astype(int)

df['explanation_size'] = df['Answer.explanation'].astype(str).apply(len)
df['Answer.option.int'] = df['Answer.option'].map({'NO': 0, 'IDK': 0.5, 'YES': 1})
df['Worker.gender.int'] = df['Worker.gender'].map({'Male': -1, 'Other': 0, 'Female': 1})
df['Worker.profession.int'] = df['Worker.profession'].map(
    {'Other': 0, 'Hobbyist': 1, 'Undergraduate_Student': 2, 'Graduate_Student': 3, 'Professional_Developer': 4})

def calculate_ttr(text):
    tokens = text.split()
    unique_tokens = set(tokens)
    ttr = len(unique_tokens) / len(tokens) if tokens else 0
    return ttr
df['TTR'] = df['Answer.explanation'].astype(str).apply(calculate_ttr)

In [4]:
# filter only correct answers
df = df[df['correctness'] == 1]

In [5]:
# group by 'FailingMethod' and 'Question.ID'
bug_groups = df.groupby(['FailingMethod', 'Question.ID'])
bug_groups.size()

FailingMethod  Question.ID
HIT01_8        0              16
               1              12
               2              18
               3              18
               4              15
                              ..
HIT08_54       124             6
               125            14
               126            10
               127             8
               128            11
Length: 129, dtype: int64

In [6]:
ground_truth = "The IF statement in line 279 checks whether minutesOffset is set to a value between 0 and 59. If not, an IllegalArgumentException is thrown. This is a bug because the minutesOffset may also be negative. The IF statement should check for the minutesOffset to be between -59 and 59."

# pick bug: HIT01_8 @ question 0
bug_group = bug_groups.get_group(('HIT01_8', 1)).copy()
bug_group

,Answer.ID,FailingMethod,Question.ID,Answer.duration,Answer.confidence,Answer.difficulty,GroundTruth,TP,TN,FN,...,Worker.gender,Worker.whereLearnedToCode,Worker.country,Worker.programmingLanguage,correctness,explanation_size,Answer.option.int,Worker.gender.int,Worker.profession.int,TTR
20,441,HIT01_8,1,140.407,4,3,1,1,0,0,...,Female,University,US,php,1,80,1.0,1,2,1.000000
22,443,HIT01_8,1,112.409,5,2,1,1,0,0,...,Male,University;Web,USA,c#,1,149,1.0,-1,2,0.931034
23,444,HIT01_8,1,76.418,5,1,1,1,0,0,...,Male,University;Web;Other FIRST Robotics,United States,C++,1,164,1.0,-1,2,0.903226
24,445,HIT01_8,1,261.928,5,2,1,1,0,0,...,Female,University,USA,java,1,550,1.0,1,3,0.770000
25,446,HIT01_8,1,236.045,5,1,1,1,0,0,...,Male,University;Other On the job,USA,Python; SQL; Java,1,233,1.0,-1,4,0.882353
27,448,HIT01_8,1,259.790,3,3,1,1,0,0,...,Female,University,united states,none,1,229,1.0,1,1,0.902439
28,449,HIT01_8,1,230.142,4,3,1,1,0,0,...,Male,High School,USA,php and C++,1,284,1.0,-1,2,0.830189
29,450,HIT01_8,1,253.302,4,2,1,1,0,0,...,Male,University;Web;Other MOOC,USA,Java Python,1,328,1.0,-1,1,0.761905
30,451,HIT01_8,1,89.521,4,3,1,1,0,0,...,Male,High School;University;Web;Other books,Russia,C++;Object Pascal,1,86,1.0,-1,4,0.937500
34,455,HIT01_8,1,161.419,4,4,1,1,0,0,...,Male,University,India,c; c++,1,144,1.0,-1,3,0.807692


In [7]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

def calculate_cosine_similarity(hyp, ref):
    hyp_embedding = model.encode(hyp)
    ref_embedding = model.encode(ref)
    return np.dot(hyp_embedding, ref_embedding) / (np.linalg.norm(hyp_embedding) * np.linalg.norm(ref_embedding))


/Users/lucas/PycharmProjects/ASE-GenAI/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
import textstat
def calculate_flesch_reading_ease(text):
    return textstat.flesch_reading_ease(text)

def calculate_automated_readability_index(text):
    return textstat.automated_readability_index(text)

In [9]:
from scipy.stats import entropy

def coefficient_variation(series, series_full):
    if len(series) < 2:
        return 0
    return series.std() / ((series_full.max() - series_full.min()) / 2)

def categorical_diversity(series):
    counts = series.value_counts(normalize=True)
    return entropy(counts, base=2)


def categorical_diversity_norm(series, series_full):
    k = series_full.nunique()
    return categorical_diversity(series) / (np.log2(k) if k > 1 else 1)

def combined_diversity_score(df_selection, df_all, categorical_cols, continuous_cols):
    total_number_cols = len(categorical_cols) + len(continuous_cols)


    categorical_score = sum(1 / total_number_cols * categorical_diversity_norm(df_selection[col], df_all[col]) for col in categorical_cols)
    continuous_score = sum(1 / total_number_cols * coefficient_variation(df_selection[col], df_all[col]) for col in continuous_cols)

    return categorical_score + continuous_score

In [10]:
# use itertools to get all possible subsets of bug_group rows
import itertools

def get_subsets(df):
    for i in range(1, len(df) + 1):
        amount = 0
        for subset in itertools.combinations(df.iterrows(), i):
            amount += 1;
            yield df.loc[[x[0] for x in subset]]
        print('n:', i, 'amount:', amount)
            
all_subsets = list(get_subsets(bug_group))

n: 1 amount: 12
n: 2 amount: 66
n: 3 amount: 220
n: 4 amount: 495
n: 5 amount: 792
n: 6 amount: 924
n: 7 amount: 792
n: 8 amount: 495
n: 9 amount: 220
n: 10 amount: 66
n: 11 amount: 12
n: 12 amount: 1


In [11]:
all_subsets[13]

,Answer.ID,FailingMethod,Question.ID,Answer.duration,Answer.confidence,Answer.difficulty,GroundTruth,TP,TN,FN,...,Worker.gender,Worker.whereLearnedToCode,Worker.country,Worker.programmingLanguage,correctness,explanation_size,Answer.option.int,Worker.gender.int,Worker.profession.int,TTR
20,441,HIT01_8,1,140.407,4,3,1,1,0,0,...,Female,University,US,php,1,80,1.0,1,2,1.000000
23,444,HIT01_8,1,76.418,5,1,1,1,0,0,...,Male,University;Web;Other FIRST Robotics,United States,C++,1,164,1.0,-1,2,0.903226


In [12]:
categorical_cols = ['Worker.profession', 'Worker.gender']
continuous_cols = ['Worker.yearsOfExperience', 'Worker.age']

In [13]:
from tqdm import tqdm

results = []
for subset in tqdm(all_subsets):
    # concat all strings of Answer.explanation in subset
    subset_text = subset['Answer.explanation'].str.cat(sep='\n\n')
    readability_score = calculate_flesch_reading_ease(subset_text)
    similarity_score = calculate_cosine_similarity(subset_text, ground_truth)
    diversity_score = combined_diversity_score(subset, bug_group, categorical_cols, continuous_cols)
    results.append((readability_score, similarity_score, diversity_score))
    
df_results = pd.DataFrame(results, columns=['readability', 'similarity', 'diversity'])
df_results
    

100%|██████████| 4095/4095 [01:21<00:00, 50.33it/s]


,readability,similarity,diversity
0,80.62,0.722904,0.000000
1,69.45,0.784441,0.000000
2,48.47,0.801706,0.000000
3,39.71,0.786183,0.000000
4,28.84,0.870643,0.000000
...,...,...,...
4090,59.33,0.837821,0.681664
4091,60.14,0.818906,0.734276
4092,59.94,0.814052,0.737129
4093,59.43,0.794701,0.655354


In [14]:
# min max normalization of each individual column 'readability', 'similarity', 'diversity'
def normalize_column(series):
    return (series - series.min()) / (series.max() - series.min())
df_results['readability.norm'] = normalize_column(df_results['readability'])
df_results['similarity.norm'] = normalize_column(df_results['similarity'])
df_results['diversity.norm'] = normalize_column(df_results['diversity'])
df_results

,readability,similarity,diversity,readability.norm,similarity.norm,diversity.norm
0,80.62,0.722904,0.000000,0.975690,0.710942,0.000000
1,69.45,0.784441,0.000000,0.876586,0.819983,0.000000
2,48.47,0.801706,0.000000,0.690445,0.850575,0.000000
3,39.71,0.786183,0.000000,0.612723,0.823071,0.000000
4,28.84,0.870643,0.000000,0.516281,0.972729,0.000000
...,...,...,...,...,...,...
4090,59.33,0.837821,0.681664,0.786798,0.914570,0.694206
4091,60.14,0.818906,0.734276,0.793985,0.881053,0.747786
4092,59.94,0.814052,0.737129,0.792210,0.872453,0.750692
4093,59.43,0.794701,0.655354,0.787685,0.838163,0.667412


In [15]:
# add normed scores
df_results['score'] = df_results['readability.norm'] + df_results['similarity.norm'] + df_results['diversity.norm']

# sort by score descending
df_results = df_results.sort_values('score', ascending=False)

df_results

,readability,similarity,diversity,readability.norm,similarity.norm,diversity.norm,score
840,66.27,0.817777,0.883779,0.848372,0.879052,0.900039,2.627463
21,77.87,0.697944,0.981933,0.951291,0.666715,1.000000,2.618006
1107,71.75,0.794625,0.863310,0.896992,0.838028,0.879194,2.614215
442,72.87,0.791136,0.847902,0.906929,0.831846,0.863503,2.602278
362,66.78,0.797721,0.887771,0.852897,0.843514,0.904105,2.600516
...,...,...,...,...,...,...,...
23,29.53,0.807400,0.026517,0.522403,0.860665,0.027004,1.410072
9,53.55,0.666195,0.000000,0.735516,0.610457,0.000000,1.345973
139,3.14,0.791545,0.190706,0.288262,0.832571,0.194215,1.315048
493,-21.57,0.755285,0.412390,0.069027,0.768321,0.419978,1.257325


In [16]:
# remove all rows where there exist a row where every indivudual score is better
df_results_reduced = df_results[~df_results.apply(lambda x: (df_results['readability.norm'] > x['readability.norm']) & (df_results['similarity.norm'] > x['similarity.norm']) & (df_results['diversity.norm'] > x['diversity.norm']), axis=1)]
df_results_reduced

,readability,similarity,diversity,readability.norm,similarity.norm,diversity.norm,score
840,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1107,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,NaN,NaN,NaN,NaN,NaN,NaN,NaN
493,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
def print_subset(i): # i is the best ranked index
    score_row = df_results.iloc[i]
    original_index = df_results.index[i]
    subset = all_subsets[original_index]
    
    print('Readability:', score_row['readability.norm'])
    print('Similarity:', score_row['similarity.norm'])
    print('Diversity:', score_row['diversity.norm'])
    print('Score:', score_row['score'])
    # print the concatted Answer.explanation
    print(subset['Answer.explanation'].str.cat(sep='\n\n'))

In [18]:
print_subset(0)
print_subset(1)

Readability: 0.848371927956703
Similarity: 0.8790521025657654
Diversity: 0.9000392350754332
Score: 2.6274632655979016
Minutes are set to -15; which is less then 0 and it throws illegal arg exception

In the code there is a check that 0 <= minutes < 60 and the minutesOffset is -15 which does not fall into these prarmeters thus throwing an Exception

YES. The issue is on line 279 (as I explained in my first question; of which I misunderstood that I was only being asked about the specific issue; not generalized issue). On line 279 the variable "minutesOffSet" is parameterized to throw an exception if it is < 0 or > 59. Line 279 should read "if (minutesOffset < -59 || minutesOffset > 59) {" because now the method can take in the number of minutes as a negative and will allow the method to properly progress to invoke/call further methods such as those asked about in the two previous questions.

This variable contains a value of -15 as set by DateTimeZone.forOffsetHoursMinutes(-2; -15). Line

In [19]:
# get the best subset's index
best_subset_index = df_results.index[0]
best_subset_index

best_subset = all_subsets[best_subset_index]
best_subset

,Answer.ID,FailingMethod,Question.ID,Answer.duration,Answer.confidence,Answer.difficulty,GroundTruth,TP,TN,FN,...,Worker.gender,Worker.whereLearnedToCode,Worker.country,Worker.programmingLanguage,correctness,explanation_size,Answer.option.int,Worker.gender.int,Worker.profession.int,TTR
20,441,HIT01_8,1,140.407,4,3,1,1,0,0,...,Female,University,US,php,1,80,1.0,1,2,1.000000
22,443,HIT01_8,1,112.409,5,2,1,1,0,0,...,Male,University;Web,USA,c#,1,149,1.0,-1,2,0.931034
24,445,HIT01_8,1,261.928,5,2,1,1,0,0,...,Female,University,USA,java,1,550,1.0,1,3,0.770000
27,448,HIT01_8,1,259.790,3,3,1,1,0,0,...,Female,University,united states,none,1,229,1.0,1,1,0.902439
35,456,HIT01_8,1,58.673,5,1,1,1,0,0,...,Male,High School;University;Other forever,USA,c++,1,215,1.0,-1,4,0.878049


In [20]:
best_subset['Answer.explanation'].str.cat(sep='\n\n')

'Minutes are set to -15; which is less then 0 and it throws illegal arg exception\n\nIn the code there is a check that 0 <= minutes < 60 and the minutesOffset is -15 which does not fall into these prarmeters thus throwing an Exception\n\nYES. The issue is on line 279 (as I explained in my first question; of which I misunderstood that I was only being asked about the specific issue; not generalized issue). On line 279 the variable "minutesOffSet" is parameterized to throw an exception if it is < 0 or > 59. Line 279 should read "if (minutesOffset < -59 || minutesOffset > 59) {" because now the method can take in the number of minutes as a negative and will allow the method to properly progress to invoke/call further methods such as those asked about in the two previous questions.\n\nThis variable contains a value of -15 as set by DateTimeZone.forOffsetHoursMinutes(-2; -15). Line 279 checks to see if is a valid value; meaning that is between 0 and 59. Since it is not; an exception error i

In [21]:
def is_pareto_efficient(costs):
    is_efficient = np.ones(costs.shape[0], dtype=bool)
    display(len(is_efficient))
    for i, c in enumerate(costs):
        if is_efficient[i]:
            is_efficient[is_efficient] = np.any(costs[is_efficient] > c, axis=1)
            is_efficient[i] = True
    return is_efficient

pareto_mask = is_pareto_efficient(df_results[["readability.norm", "similarity.norm", "diversity.norm"]].values)
pareto_df = df_results[pareto_mask]
pareto_df

4095

,readability,similarity,diversity,readability.norm,similarity.norm,diversity.norm,score
840,66.27,0.817777,0.883779,0.848372,0.879052,0.900039,2.627463
21,77.87,0.697944,0.981933,0.951291,0.666715,1.000000,2.618006
1107,71.75,0.794625,0.863310,0.896992,0.838028,0.879194,2.614215
442,72.87,0.791136,0.847902,0.906929,0.831846,0.863503,2.602278
362,66.78,0.797721,0.887771,0.852897,0.843514,0.904105,2.600516
...,...,...,...,...,...,...,...
556,72.97,0.790803,0.501181,0.907817,0.831257,0.510402,2.249476
293,75.81,0.736891,0.514847,0.933014,0.735727,0.524319,2.193060
192,48.64,0.886034,0.258099,0.691953,1.000000,0.262847,1.954800
0,80.62,0.722904,0.000000,0.975690,0.710942,0.000000,1.686632


In [28]:
from scipy.optimize import minimize

# Hyperboloid parameters (you can adjust these as needed)
a = 1  # Semi-axis for readability.norm (x-axis)
b = 1  # Semi-axis for similarity.norm (y-axis)
c = 1  # Semi-axis for diversity.norm (z-axis)

# Define the objective function for the optimization
# Objective: Maximize z (diversity.norm)
def objective(vars):
    x, y, z = vars
    return -z  # We want to maximize z, so minimize the negative of z

# Constraints based on the hyperboloid equation: x^2/a^2 + y^2/b^2 - z^2/c^2 = 1
def constraint(vars):
    x, y, z = vars
    return (x**2 / a**2 + y**2 / b**2 - z**2 / c**2) - 1

# Optimization function using the scipy minimize
def find_peak(df):
    # Extract the columns as initial guesses (take the first row as an example)
    initial_guess = [df["readability.norm"].iloc[0],
                     df["similarity.norm"].iloc[0],
                     df["diversity.norm"].iloc[0]]

    # Set up the constraint dictionary for the hyperboloid equation
    con = {'type': 'eq', 'fun': constraint}

    # Perform the optimization using scipy's minimize function
    result = minimize(objective, initial_guess, constraints=con)

    # Extract the optimized values
    optimized_x, optimized_y, optimized_z = result.x
    return optimized_x, optimized_y, optimized_z

# Find the peak
optimized_x, optimized_y, optimized_z = find_peak(pareto_df[["readability.norm", "similarity.norm", "diversity.norm"]])

# Display the peak
print(f"Optimized coordinates: x = {optimized_x}, y = {optimized_y}, z = {optimized_z}")


Optimized coordinates: x = 2148628045861073.0, y = 2233542330055644.0, z = 3099246934855179.0


In [ ]:
import matplotlib
matplotlib.use('MacOSX')

In [31]:
import matplotlib.pyplot as plt

color_values = pareto_df["readability.norm"] + pareto_df["similarity.norm"] + pareto_df["diversity.norm"]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(pareto_df["readability.norm"],
           pareto_df["similarity.norm"],
           pareto_df["diversity.norm"],
           c=color_values,
           cmap='plasma',
           label='Pareto-optimal points')

ax.set_xlabel('Readability')
ax.set_ylabel('Similarity')
ax.set_zlabel('Diversity')
plt.legend()
plt.show()

In [23]:
# Pick subset of answers

# Objectives
# (Minimize k amount of explanations, maybe dont by entropy maximization)
# Maximize subset with regard to:
# - Sum Similarity
# - Sum Readability
# - Max subset entropy/diversity

In [24]:
# Idea: Greedy until taking another element into the set does not increase the objective function